# Feature Selection with Categorical Features

이 노트북은 **수치형 + FE + 범주형(원-핫 인코딩)** 을 모두 포함했을 때
F1 / AUC가 어떻게 달라지는지 다시 검증하는 용도입니다.

- 입력 데이터: `../data/raw_data.csv`
- 기준: `FE_validation.ipynb`에서 정리된 **base_num + 핵심 FE 세트**를 재사용
- 추가: `gender`, `country`, `subscription_type`, `device_type` 등 **범주형을 One-Hot 인코딩**해서 포함

여기서 얻고 싶은 것:
- **수치형 + FE만 쓴 경우 vs 수치형 + FE + 범주형까지 쓴 경우**의 F1 / AUC 비교
- 범주형이 실제로 **성능에 얼마나 기여하는지**를 수치로 확인

In [2]:
# 1. 데이터 로드 및 FE 생성 (FE_validation.ipynb 로직 재사용 버전)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score

pd.set_option("display.max_columns", None)

DATA_PATH = "../data/raw_data.csv"

df = pd.read_csv(DATA_PATH)
print("df shape =", df.shape)
print(df.columns)

# 수치형 기본 컬럼 (base_num)
BASE_NUM_COLS = [
    "age",
    "listening_time",
    "songs_played_per_day",
    "skip_rate",
    "ads_listened_per_week",
    "offline_listening",
]

# 원본 범주형 컬럼
RAW_CAT_COLS = [
    "gender",
    "country",
    "subscription_type",
    "device_type",
]


def make_fe_dataframe_full() -> pd.DataFrame:
    """FE_validation.ipynb에서 사용한 FE들을 통합 생성하는 함수
    - 결측치 median 처리 (수치형)
    - FE 4개: engagement_score, listening_time_bin, skip_rate_cap, ads_pressure
    - 추가 FE: songs_per_minute, skip_intensity
    - 추가 범주형 기반 FE: subscription_type_level, age_group, ads_listened_log
    (원본 범주형 컬럼은 그대로 유지해서 나중에 One-Hot 인코딩에 사용)
    """
    df_tmp = df.copy()

    # 1) 수치형 결측치 median 처리
    for c in BASE_NUM_COLS:
        if c in df_tmp.columns and df_tmp[c].isnull().any():
            df_tmp[c] = df_tmp[c].fillna(df_tmp[c].median())

    # 2) FE 4개 생성 (engagement_score, listening_time_bin, skip_rate_cap, ads_pressure)
    # 2-1) engagement_score
    if {"listening_time", "songs_played_per_day"}.issubset(df_tmp.columns):
        df_tmp["engagement_score"] = (
            df_tmp["listening_time"] * df_tmp["songs_played_per_day"]
        )

    # 2-2) listening_time_bin
    if "listening_time" in df_tmp.columns:
        try:
            df_tmp["listening_time_bin"] = pd.qcut(
                df_tmp["listening_time"], 3, labels=["low", "mid", "high"]
            )
        except Exception:
            bins = [0, 60, 180, df_tmp["listening_time"].max()]
            df_tmp["listening_time_bin"] = pd.cut(
                df_tmp["listening_time"],
                bins=bins,
                labels=["low", "mid", "high"],
                include_lowest=True,
            )

    # 2-3) skip_rate_cap
    if "skip_rate" in df_tmp.columns:
        df_tmp["skip_rate_cap"] = df_tmp["skip_rate"].clip(lower=0, upper=1.5)

    # 2-4) ads_pressure
    if {"ads_listened_per_week", "listening_time"}.issubset(df_tmp.columns):
        lt_nonzero = df_tmp["listening_time"].replace(0, np.nan)
        df_tmp["ads_pressure"] = df_tmp["ads_listened_per_week"] / lt_nonzero

    # 3) 추가 FE: songs_per_minute, skip_intensity (Set D 기준)
    lt_safe = df_tmp["listening_time"].replace(0, np.nan)
    df_tmp["songs_per_minute"] = (
        df_tmp["songs_played_per_day"] / lt_safe
    ).fillna(0.0)
    df_tmp["skip_intensity"] = df_tmp["skip_rate"] * df_tmp["songs_played_per_day"]

    # 4) 범주형 기반 FE (v3 기준)
    # 4-1) subscription_type_level (순서형 매핑)
    if "subscription_type" in df_tmp.columns:
        level_map = {
            "Free": 0,
            "Student": 1,
            "Premium": 2,
            "Family": 3,
        }
        df_tmp["subscription_type_level"] = (
            df_tmp["subscription_type"].map(level_map).fillna(-1).astype(int)
        )

    # 4-2) age_group (카테고리 구간)
    if "age" in df_tmp.columns:
        bins = [0, 24, 34, 44, df_tmp["age"].max()]
        labels = ["young", "adult", "middle", "senior"]
        df_tmp["age_group"] = pd.cut(
            df_tmp["age"],
            bins=bins,
            labels=labels,
            include_lowest=True,
            right=True,
        )

    # 4-3) ads_listened_log (로그 변환)
    if "ads_listened_per_week" in df_tmp.columns:
        df_tmp["ads_listened_log"] = np.log1p(df_tmp["ads_listened_per_week"])

    return df_tmp


df_fe = make_fe_dataframe_full()
print("\nFE 생성 후 컬럼 예시:")
print(df_fe.columns)

df shape = (8000, 12)
Index(['user_id', 'gender', 'age', 'country', 'subscription_type',
       'listening_time', 'songs_played_per_day', 'skip_rate', 'device_type',
       'ads_listened_per_week', 'offline_listening', 'is_churned'],
      dtype='object')

FE 생성 후 컬럼 예시:
Index(['user_id', 'gender', 'age', 'country', 'subscription_type',
       'listening_time', 'songs_played_per_day', 'skip_rate', 'device_type',
       'ads_listened_per_week', 'offline_listening', 'is_churned',
       'engagement_score', 'listening_time_bin', 'skip_rate_cap',
       'ads_pressure', 'songs_per_minute', 'skip_intensity',
       'subscription_type_level', 'age_group', 'ads_listened_log'],
      dtype='object')


In [5]:
# 2. 수치형만 vs 수치형+FE+범주형(One-Hot) F1 / AUC 비교

# (1) 타깃 정의
y = df_fe["is_churned"]

# (2) 수치형 + 핵심 FE 세트 (기존 Set D / 추천 세트 기반)
NUM_FE_COLS = [
    "engagement_score",
    "songs_per_minute",
    "skip_intensity",
    "skip_rate_cap",
]

# ads_pressure, ads_listened_log 정도는 보조로 추가해도 크게 문제 없음
OPTIONAL_FE_COLS = [
    "ads_pressure",
    "ads_listened_log",
]

num_cols_exist = [c for c in BASE_NUM_COLS if c in df_fe.columns]
num_fe_exist = [c for c in NUM_FE_COLS if c in df_fe.columns]
opt_fe_exist = [c for c in OPTIONAL_FE_COLS if c in df_fe.columns]

X_num_only = df_fe[num_cols_exist + num_fe_exist]  # 기존 우리가 주로 쓰던 세트와 유사

# (3) 범주형 + 구간형 컬럼들 One-Hot 인코딩
cat_cols = []

# 원본 범주형
for c in RAW_CAT_COLS:
    if c in df_fe.columns:
        cat_cols.append(c)

# 파생 범주형(listening_time_bin, age_group)
for c in ["listening_time_bin", "age_group"]:
    if c in df_fe.columns:
        cat_cols.append(c)

print("\nOne-Hot 인코딩 대상 범주형 컬럼:", cat_cols)

if cat_cols:
    X_cat = pd.get_dummies(df_fe[cat_cols], drop_first=True)
else:
    X_cat = pd.DataFrame(index=df_fe.index)

# 수치형 + FE + 범주형 모두 포함한 세트
X_all = pd.concat([X_num_only, df_fe[opt_fe_exist], X_cat], axis=1)

print("X_num_only shape:", X_num_only.shape)
print("X_all shape:", X_all.shape)


def evaluate_rf_best_f1(X, y, thresholds=None):
    """RandomForest + threshold 튜닝으로 best F1 / AUC 계산"""
    if thresholds is None:
        thresholds = np.linspace(0.1, 0.9, 17)

    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=5,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42,
    )
    rf.fit(X_train, y_train)

    y_proba = rf.predict_proba(X_valid)[:, 1]

    best_f1 = 0.0
    best_th = 0.5
    for th in thresholds:
        y_pred_th = (y_proba >= th).astype(int)
        f1 = f1_score(y_valid, y_pred_th)
        if f1 > best_f1:
            best_f1 = f1
            best_th = th

    auc = roc_auc_score(y_valid, y_proba)
    return best_f1, best_th, auc


print("\n=== RF + best F1 기준 성능 비교 (수치형 vs 수치형+범주형) ===")

best_f1_num, best_th_num, auc_num = evaluate_rf_best_f1(X_num_only, y)
print("[Numeric only]   best F1 = {:.4f} @ th={:.2f}, AUC = {:.4f}".format(best_f1_num, best_th_num, auc_num))

best_f1_all, best_th_all, auc_all = evaluate_rf_best_f1(X_all, y)
print("[Numeric + cat] best F1 = {:.4f} @ th={:.2f}, AUC = {:.4f}".format(best_f1_all, best_th_all, auc_all))

# (선택) 중요도 상위 몇 개만 확인해보고 싶으면 아래 주석을 풀어서 사용
# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_all, y, test_size=0.2, random_state=42, stratify=y
# )
# rf_all = RandomForestClassifier(
#     n_estimators=300,
#     max_depth=None,
#     min_samples_split=5,
#     class_weight="balanced",
#     n_jobs=-1,
#     random_state=42,
# )
# rf_all.fit(X_train, y_train)
# importances = rf_all.feature_importances_
# feat_names = X_all.columns
# imp_df = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values(
#     "importance", ascending=False
# )
# imp_df.head(20)



One-Hot 인코딩 대상 범주형 컬럼: ['gender', 'country', 'subscription_type', 'device_type', 'listening_time_bin', 'age_group']
X_num_only shape: (8000, 10)
X_all shape: (8000, 31)

=== RF + best F1 기준 성능 비교 (수치형 vs 수치형+범주형) ===
[Numeric only]   best F1 = 0.4127 @ th=0.10, AUC = 0.5289
[Numeric + cat] best F1 = 0.4113 @ th=0.10, AUC = 0.5117


---

## 3. K-Fold 기반 RF + Threshold 튜닝 (수치형 + FE 기준)

여기서는 **수치형 + 핵심 FE 세트(`X_num_only`)**를 기준으로,
K-Fold 교차검증으로 threshold까지 튜닝한 **보다 안정적인 F1/AUC**를 계산합니다.

- 모델: `RandomForestClassifier(class_weight="balanced")`
- CV: `StratifiedKFold(n_splits=5)`
- Threshold 탐색: 0.05 ~ 0.35 구간에서 0.01 간격

이 결과가 사실상 **이 데이터에서 낼 수 있는 상한선에 어느 정도 근접한 F1**이라고 보면 됩니다.


In [6]:
# 3-1. K-Fold 기반 RF + threshold 튜닝 함수 정의

from sklearn.model_selection import StratifiedKFold


def evaluate_model_cv_best_f1(model, X, y, thresholds=None, n_splits=5):
    """아무 분류 모델이나 받아서, K-Fold CV 기반으로 best F1 / AUC 계산
    - model: scikit-learn 분류기 (fit/predict_proba 지원)
    - thresholds: 탐색할 threshold 리스트 (None이면 0.05~0.35, step=0.01)
    - n_splits: StratifiedKFold 분할 개수
    """
    if thresholds is None:
        thresholds = np.round(np.arange(0.05, 0.36, 0.01), 2)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    all_y_true = []
    all_y_score = []

    for train_idx, valid_idx in skf.split(X, y):
        X_tr, X_vl = X.iloc[train_idx], X.iloc[valid_idx]
        y_tr, y_vl = y.iloc[train_idx], y.iloc[valid_idx]

        mdl = model  # 원본 모델을 그대로 쓰면 fold 간 누적 학습 위험 → clone 사용 권장
        from sklearn.base import clone

        mdl = clone(model)
        mdl.fit(X_tr, y_tr)

        y_proba = mdl.predict_proba(X_vl)[:, 1]

        all_y_true.append(y_vl)
        all_y_score.append(y_proba)

    all_y_true = np.concatenate([arr.values if hasattr(arr, "values") else arr for arr in all_y_true])
    all_y_score = np.concatenate(all_y_score)

    # threshold별 F1 측정
    best_f1 = 0.0
    best_th = 0.5
    for th in thresholds:
        y_pred_th = (all_y_score >= th).astype(int)
        f1 = f1_score(all_y_true, y_pred_th)
        if f1 > best_f1:
            best_f1 = f1
            best_th = float(th)

    auc = roc_auc_score(all_y_true, all_y_score)
    return best_f1, best_th, auc


# 3-2. 기본 RF 모델로 CV 기반 best F1 / AUC 계산 (수치형 + 핵심 FE 기준)

base_rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=5,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42,
)

best_f1_cv, best_th_cv, auc_cv = evaluate_model_cv_best_f1(base_rf, X_num_only, y)

print("=== RF (Numeric + 핵심 FE, K-Fold 기반) ===")
print("best F1 (CV) = {:.4f} @ th={:.2f}, AUC (CV) = {:.4f}".format(best_f1_cv, best_th_cv, auc_cv))


=== RF (Numeric + 핵심 FE, K-Fold 기반) ===
best F1 (CV) = 0.4120 @ th=0.11, AUC (CV) = 0.5280


---

## 4. RandomizedSearchCV로 RF 하이퍼파라미터 튜닝

이제 같은 `X_num_only`(수치형 + 핵심 FE)에 대해 **RF 하이퍼파라미터를 랜덤 탐색**으로 튜닝합니다.

- 튜닝 대상:
  - `n_estimators`: [200, 300, 400, 500, 600]
  - `max_depth`: [3, 5, 7, None]
  - `min_samples_split`: [2, 5, 10]
  - `min_samples_leaf`: [1, 2, 5]
  - `max_features`: ["sqrt", "log2", 0.5]
- CV: 3-Fold, scoring = `f1`

튜닝된 RF를 다시 **K-Fold + threshold 튜닝**으로 평가해서,
기본 RF 대비 F1/AUC가 얼마나 개선되는지 비교합니다.


In [7]:
# 4-1. RF RandomizedSearchCV 수행

from sklearn.model_selection import RandomizedSearchCV

rf_for_search = RandomForestClassifier(
    class_weight="balanced",
    n_jobs=-1,
    random_state=42,
)

param_dist = {
    "n_estimators": [200, 300, 400, 500, 600],
    "max_depth": [3, 5, 7, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "max_features": ["sqrt", "log2", 0.5],
}

rf_random = RandomizedSearchCV(
    estimator=rf_for_search,
    param_distributions=param_dist,
    n_iter=25,  # 시간이 너무 오래 걸리면 15~20으로 줄여도 됨
    scoring="f1",
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1,
)

print("RandomizedSearchCV (RF) fitting...")
rf_random.fit(X_num_only, y)

print("\n=== RF RandomizedSearchCV best params ===")
print(rf_random.best_params_)
print("best CV F1 (search scoring) = {:.4f}".format(rf_random.best_score_))

# 4-2. 튜닝된 RF를 K-Fold + threshold 튜닝으로 재평가

best_rf = rf_random.best_estimator_

best_f1_cv_tuned, best_th_cv_tuned, auc_cv_tuned = evaluate_model_cv_best_f1(
    best_rf, X_num_only, y
)

print("\n=== Tuned RF (Numeric + 핵심 FE, K-Fold 기반) ===")
print(
    "best F1 (CV) = {:.4f} @ th={:.2f}, AUC (CV) = {:.4f}".format(
        best_f1_cv_tuned, best_th_cv_tuned, auc_cv_tuned
    )
)

print("\n[비교] 기본 RF vs 튜닝 RF (둘 다 X_num_only 기준)")
print(
    "- 기본 RF : F1 = {:.4f}, AUC = {:.4f}".format(
        best_f1_cv, auc_cv
    )
)
print(
    "- 튜닝 RF : F1 = {:.4f}, AUC = {:.4f}".format(
        best_f1_cv_tuned, auc_cv_tuned
    )
)


RandomizedSearchCV (RF) fitting...
Fitting 3 folds for each of 25 candidates, totalling 75 fits

=== RF RandomizedSearchCV best params ===
{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 3}
best CV F1 (search scoring) = 0.3313

=== Tuned RF (Numeric + 핵심 FE, K-Fold 기반) ===
best F1 (CV) = 0.4113 @ th=0.05, AUC (CV) = 0.5071

[비교] 기본 RF vs 튜닝 RF (둘 다 X_num_only 기준)
- 기본 RF : F1 = 0.4120, AUC = 0.5280
- 튜닝 RF : F1 = 0.4113, AUC = 0.5071


---

## 5. (선택) 튜닝 RF + HGB + LR 간단 앙상블

시간 여유가 있으면, 아래 셀로 **간단 소프트 보팅 앙상블**도 시도해 볼 수 있습니다.

- 구성 예시:
  - 튜닝된 RF (`best_rf`)
  - `HistGradientBoostingClassifier`
  - `LogisticRegression` (수치형 + FE만 사용)

이 셀은 **필요할 때만 실행**해도 되고, 너무 무겁게 느껴지면 건너뛰어도 괜찮습니다.


In [8]:
# 5-1. 간단 소프트 보팅 앙상블 (선택 실행)

from sklearn.ensemble import HistGradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

# HGB / LR 기본 세팅 (필요하면 나중에 따로 튜닝 가능)
hgb_clf = HistGradientBoostingClassifier(random_state=42)

lr_clf = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1,
)

# VotingClassifier 구성 (모두 X_num_only 기준)
ensemble = VotingClassifier(
    estimators=[
        ("rf", best_rf),
        ("hgb", hgb_clf),
        ("lr", lr_clf),
    ],
    voting="soft",
    n_jobs=-1,
)

best_f1_ens, best_th_ens, auc_ens = evaluate_model_cv_best_f1(
    ensemble, X_num_only, y
)

print("=== Soft Voting Ensemble (RF + HGB + LR, X_num_only) ===")
print(
    "best F1 (CV) = {:.4f} @ th={:.2f}, AUC (CV) = {:.4f}".format(
        best_f1_ens, best_th_ens, auc_ens
    )
)

print("\n[비교] 기본 RF vs 튜닝 RF vs 앙상블")
print("- 기본 RF   : F1 = {:.4f}, AUC = {:.4f}".format(best_f1_cv, auc_cv))
print("- 튜닝 RF   : F1 = {:.4f}, AUC = {:.4f}".format(best_f1_cv_tuned, auc_cv_tuned))
print("- 앙상블 RF: F1 = {:.4f}, AUC = {:.4f}".format(best_f1_ens, auc_ens))


=== Soft Voting Ensemble (RF + HGB + LR, X_num_only) ===
best F1 (CV) = 0.4113 @ th=0.29, AUC (CV) = 0.5169

[비교] 기본 RF vs 튜닝 RF vs 앙상블
- 기본 RF   : F1 = 0.4120, AUC = 0.5280
- 튜닝 RF   : F1 = 0.4113, AUC = 0.5071
- 앙상블 RF: F1 = 0.4113, AUC = 0.5169


---

## 6. 피처별 이탈 영향도 분석 (왜 성능이 안 나오는지 원인 파악)

지금까지 **모든 튜닝/FE/앙상블을 해도 F1이 0.41대에서 막혔다**는 것은,
**데이터 자체의 한계** 때문일 가능성이 크다.

여기서는 각 피처가 `is_churned`와 **실제로 얼마나 연관이 있는지**를 확인해서,
**"왜 성능이 안 올라가는가?"**의 근본 원인을 찾아본다.

### 분석 목표
1. **수치형 피처**: 이탈 vs 비이탈 분포 비교 (박스플롯, t-test p-value)
2. **범주형 피처**: 각 카테고리별 이탈률 차이
3. **상관계수**: 피처 vs is_churned (Point-Biserial Correlation)
4. **RF Feature Importance + Permutation Importance** (통합 확인)


In [11]:
# 6-1. 수치형 피처: 이탈 vs 비이탈 분포 비교 (t-test p-value 포함)

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 수치형 기본 + 핵심 FE
numeric_features = BASE_NUM_COLS + NUM_FE_COLS

churned = df_fe[df_fe["is_churned"] == 1]
not_churned = df_fe[df_fe["is_churned"] == 0]

print("=== 수치형 피처별 이탈 vs 비이탈 분포 비교 (t-test) ===\n")

ttest_results = []

for col in numeric_features:
    if col not in df_fe.columns:
        continue
    
    # t-test 수행
    val_churned = churned[col].dropna()
    val_not_churned = not_churned[col].dropna()
    
    t_stat, p_val = stats.ttest_ind(val_churned, val_not_churned, equal_var=False)
    
    mean_churned = val_churned.mean()
    mean_not_churned = val_not_churned.mean()
    
    ttest_results.append({
        "feature": col,
        "mean_churned": mean_churned,
        "mean_not_churned": mean_not_churned,
        "diff": mean_churned - mean_not_churned,
        "t_stat": t_stat,
        "p_value": p_val,
    })

ttest_df = pd.DataFrame(ttest_results).sort_values("p_value")
print(ttest_df.to_string(index=False))

# p-value가 작을수록(< 0.05) 이탈과 유의미한 관계
print("\n▶ p-value < 0.05인 피처들만 이탈과 유의미한 차이가 있음")
print("▶ p-value가 크면 → 이탈 여부와 거의 무관 → 모델 성능에 기여 어려움")


=== 수치형 피처별 이탈 vs 비이탈 분포 비교 (t-test) ===

              feature  mean_churned  mean_not_churned      diff    t_stat  p_value
        skip_rate_cap      0.331378          0.320691  0.010687  1.726153 0.084408
            skip_rate      0.368289          0.351054  0.017235  1.430528 0.152658
    offline_listening      0.757122          0.744476  0.012646  1.149979 0.250227
 songs_played_per_day     50.617576         49.992073  0.625503  0.876739 0.380687
       skip_intensity     18.404178         17.770269  0.633909  0.869820 0.384456
       listening_time    153.174312        154.536347 -1.362035 -0.648440 0.516741
     songs_per_minute      0.607387          0.595543  0.011844  0.499394 0.617533
                  age     38.998551         38.811941  0.186610  0.418242 0.675795
ads_listened_per_week      6.891357          6.962220 -0.070863 -0.202819 0.839288
     engagement_score   7724.631579       7698.635689 25.995890  0.158765 0.873863

▶ p-value < 0.05인 피처들만 이탈과 유의미한 차이가 있음
▶ p-v

In [12]:
# 6-2. 범주형 피처: 카테고리별 이탈률 차이

print("\n=== 범주형 피처별 이탈률 ===\n")

cat_features = RAW_CAT_COLS + ["listening_time_bin", "age_group"]

for col in cat_features:
    if col not in df_fe.columns:
        continue
    
    churn_by_cat = df_fe.groupby(col)["is_churned"].agg(["mean", "count"])
    churn_by_cat.columns = ["churn_rate", "count"]
    churn_by_cat = churn_by_cat.sort_values("churn_rate", ascending=False)
    
    print(f"\n[{col}]")
    print(churn_by_cat.to_string())
    
print("\n▶ 카테고리 간 이탈률 차이가 크지 않으면 → 범주형이 이탈 예측에 별 도움 안 됨")



=== 범주형 피처별 이탈률 ===


[gender]
        churn_rate  count
gender                   
Female    0.262881   2659
Other     0.261887   2650
Male      0.251951   2691

[country]
         churn_rate  count
country                   
PK         0.275275    999
DE         0.272906   1015
FR         0.271992    989
AU         0.257253   1034
US         0.253876   1032
CA         0.248428    954
UK         0.247412    966
IN         0.243323   1011

[subscription_type]
                   churn_rate  count
subscription_type                   
Family               0.275157   1908
Student              0.261868   1959
Premium              0.250591   2115
Free                 0.249257   2018

[device_type]
             churn_rate  count
device_type                   
Mobile         0.268950   2599
Desktop        0.257379   2778
Web            0.250477   2623

[listening_time_bin]
                    churn_rate  count
listening_time_bin                   
low                   0.261567   2680
mid     

C:\Users\jiyon\AppData\Local\Temp\ipykernel_24192\4263521319.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  churn_by_cat = df_fe.groupby(col)["is_churned"].agg(["mean", "count"])
C:\Users\jiyon\AppData\Local\Temp\ipykernel_24192\4263521319.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  churn_by_cat = df_fe.groupby(col)["is_churned"].agg(["mean", "count"])


In [13]:
# 6-3. 상관계수: 피처 vs is_churned (Point-Biserial Correlation)

from scipy.stats import pointbiserialr

print("\n=== 피처 vs is_churned 상관계수 (Point-Biserial) ===\n")

corr_results = []

for col in numeric_features:
    if col not in df_fe.columns:
        continue
    
    # Point-Biserial Correlation (연속형 vs 이진)
    vals = df_fe[col].dropna()
    target = df_fe.loc[vals.index, "is_churned"]
    
    corr, p_val = pointbiserialr(target, vals)
    
    corr_results.append({
        "feature": col,
        "correlation": corr,
        "p_value": p_val,
    })

corr_df = pd.DataFrame(corr_results).sort_values("correlation", key=abs, ascending=False)
print(corr_df.to_string(index=False))

print("\n▶ |correlation|이 0.1 미만이면 → 이탈과 거의 선형 관계 없음")
print("▶ 모든 피처의 상관계수가 약하면 → 단순 선형 모델로는 예측 한계")
print("▶ RF도 비선형 조합을 찾지 못하면 → F1이 낮게 나올 수밖에 없음")



=== 피처 vs is_churned 상관계수 (Point-Biserial) ===

              feature  correlation  p_value
        skip_rate_cap     0.019652 0.078815
            skip_rate     0.016598 0.137689
    offline_listening     0.012754 0.254029
 songs_played_per_day     0.009776 0.381956
       skip_intensity     0.009734 0.384001
       listening_time    -0.007217 0.518678
     songs_per_minute     0.005664 0.612463
                  age     0.004666 0.676458
ads_listened_per_week    -0.002279 0.838474
     engagement_score     0.001781 0.873438

▶ |correlation|이 0.1 미만이면 → 이탈과 거의 선형 관계 없음
▶ 모든 피처의 상관계수가 약하면 → 단순 선형 모델로는 예측 한계
▶ RF도 비선형 조합을 찾지 못하면 → F1이 낮게 나올 수밖에 없음


In [14]:
# 6-4. RF Feature Importance + Permutation Importance 통합 확인

from sklearn.inspection import permutation_importance

# 기본 RF 모델로 학습 (X_num_only 기준)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_num_only, y, test_size=0.2, random_state=42, stratify=y
)

rf_for_imp = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=5,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42,
)
rf_for_imp.fit(X_train, y_train)

# (1) Feature Importance
feat_imp = rf_for_imp.feature_importances_
feat_names = X_num_only.columns

imp_df = pd.DataFrame({"feature": feat_names, "importance": feat_imp})
imp_df = imp_df.sort_values("importance", ascending=False)

print("\n=== RF Feature Importance (X_num_only) ===\n")
print(imp_df.to_string(index=False))

# (2) Permutation Importance
perm_imp = permutation_importance(
    rf_for_imp, X_valid, y_valid, n_repeats=10, random_state=42, scoring="f1"
)

perm_imp_df = pd.DataFrame({
    "feature": feat_names,
    "perm_importance_mean": perm_imp.importances_mean,
    "perm_importance_std": perm_imp.importances_std,
})
perm_imp_df = perm_imp_df.sort_values("perm_importance_mean", ascending=False)

print("\n=== Permutation Importance (F1 기준, X_num_only) ===\n")
print(perm_imp_df.to_string(index=False))

print("\n▶ Feature Importance가 모두 비슷하면 → 단일 피처로 강력한 예측 불가")
print("▶ Permutation Importance가 낮으면 → 실제 성능에 기여 적음")
print("▶ 두 지표가 모두 낮으면 → 데이터 자체의 예측 신호가 약함")



=== RF Feature Importance (X_num_only) ===

              feature  importance
     songs_per_minute    0.143454
     engagement_score    0.141516
       skip_intensity    0.134431
       listening_time    0.129051
                  age    0.124598
 songs_played_per_day    0.104091
            skip_rate    0.085738
        skip_rate_cap    0.083754
ads_listened_per_week    0.042155
    offline_listening    0.011214

=== Permutation Importance (F1 기준, X_num_only) ===

              feature  perm_importance_mean  perm_importance_std
     songs_per_minute              0.076790             0.006422
 songs_played_per_day              0.073286             0.008196
       listening_time              0.069356             0.008617
     engagement_score              0.068503             0.007189
       skip_intensity              0.066470             0.006152
            skip_rate              0.066110             0.007591
        skip_rate_cap              0.061881             0.009809
ads_list

---

## 7. 최종 진단 요약

위 섹션 6의 분석 결과를 종합하면, **"왜 F1이 0.41대에서 막히는가?"**의 근본 원인을 찾을 수 있습니다.

### 예상되는 원인들 (분석 후 확인)

1. **피처 vs 이탈 간 상관이 약함**  
   - 모든 수치형 피처의 상관계수가 |0.1| 미만이면 → 단순 선형으로는 예측 불가  
   - RF도 비선형 조합을 못 찾으면 → 성능 한계

2. **범주형 피처의 이탈률 차이가 작음**  
   - 예: Free vs Premium 이탈률 차이가 5%p 이내면 → 범주형 추가해도 성능 향상 미미

3. **Feature Importance가 모두 고르게 분산**  
   - 특정 피처가 압도적으로 중요하지 않고, 10개가 골고루 10% 내외씩 → **단일 강력 예측 변수 없음**

4. **데이터 구조 자체의 한계**  
   - 유저당 1행 스냅샷 → 시계열 정보 없음  
   - 이탈 직전의 급격한 행동 변화를 포착 불가  
   - 결과적으로 **"이탈 예측 신호"가 약한 데이터 구조**

### 대응 방향

- **현재 F1 0.41 ~ 0.42**를 "이 데이터에서의 현실적 상한"으로 보고,  
  → 발표 때는 **"데이터 한계 + 추가 데이터 필요성"**을 중심으로 스토리 구성  
- 추가 성능 향상보다는 **"왜 한계인가에 대한 근거 기반 설명"**이 더 설득력 있음
